### Финальный проект студента- Страшко Петр
### Тема - Бот справочник для ориентации в Minecraft 1.7.10 моде: Ars Magica 2
### Для проверки можно использовать бота по адресу - https://t.me/ars_grimoire_bot или же @ars_grimoire_bot

##### 1) Практическая применимость 
Бот нужен для быстрой ориентации в заклинаниях мода Ars Magica 2, это очень сложно, ведь нормальных тектовых гайдов по моду я не нашел.

##### 2) Понятный интерфейс 
Пример: В поиске по названию заклинания бот явно указывает что или заклинание не найдено, или неверный ввод и указывает, что можно ввести еще раз.

##### 3) Условия
Условия в коде встречаются часто. Например в функции, которая создает словарь из заклинаний (get_listing) идет проверка на типы кнопок, а также на страницу типов кнопок, там используется также "and"

##### 4) Цикл
Цикл for используется в функции для создания клавиатуры из кнопок, а кнопки из каждого элемента словаря (который берется из get_listing). Мы перебираем все записи в словаре и делаем из них кнопки, добавляя в список кнопок, а потом из списка билдер, а из билдера клавиатуру.

##### 5) Словари и множества
Все результаты вывода из базы данных представляют из себя список кортежей, словарь же используется KNOPKI, где каждая запись - инфа для кнопок. Ключ - callback_data а значение - Тект кнопки

##### 6) Запись в файл
Бот записывает телеграмм айди пользователя и хранит его в базе данных, вместе с типом просматриваемых заклинаний и страницей просмотра.

##### 7) Комментарии в коде
Их много )

##### 8) Правильность кода на любом запуске
Все должно работать хорошо, поиск работает отлично, а в остальных случаях нужно лишь жмакать кнопки.

##### 9) Своя функция
Свои функции написаны и широко используются

##### 10) Новая функция
wget - для загрузки базы данных.
nest_asyncio - для предотвращения ошибок с jupyter
aiogram - для работы телеграмм бота
sqlite3 - самая простая база данных

In [2]:
#Качаем все нужные библиотеки
!pip install nest_asyncio
!pip install aiogram==3.3.0
!pip install wget
import wget
#Качаем базу данных
wget.download("https://vk.com/doc275457500_670117305?hash=LkSzpq5N2vMFIFBFZFuMpw99gJwiCzzsuRqNTRhPvkz&dl=7Tsp2hjbqBqqxLecenXJPRZsJzFt5zu9ZWZ8xzZD94w", "ars_database.db")

In [ ]:
import nest_asyncio
import sqlite3
import wget
from aiogram import Bot, Dispatcher, F
from aiogram.filters import Command, CommandStart, StateFilter
from aiogram.filters.callback_data import CallbackData
from aiogram.types import CallbackQuery, InlineKeyboardButton, InlineKeyboardMarkup, Message
from aiogram.fsm.state import State, StatesGroup
from aiogram.fsm.context import FSMContext
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram.utils.keyboard import InlineKeyboardBuilder

#В курсе по телеграмм ботам на степик сказали, что Jupyter работает с aiogram только так.
nest_asyncio.apply()
#Токен бота, получается от Bot_Father, никому не передавать.
token = '6559256220:AAH2seyNnX07za15mxp-iC8aS5UZgfQDDC4'

storage = MemoryStorage()
#Инициализация бота и диспетчера (Штука, которая отлавливает новости от телеграмм и распределяет для каждой новости свою функцию)
bot = Bot(token)
dp = Dispatcher()

#Эта штука создает callback_data из разных данных, объединяя их, нужно для создания кнопок из базы.
class SpellsCallbackFactory(CallbackData, prefix="spell"):
    spell_id: int
    name: str
#Тут состояние ожидание ввода поиска.
class FSMFillForm(StatesGroup):
    search  = State()

#Словарь из кнопок, ключ - callback_data, значение - текст кнопки.
KNOPKI: dict[str, str] = {
    'button1': 'Все спелы ⭐️⭐️⭐️',
    'button2': 'Все формы 🟨🟨🟨',
    'button3': 'Все компоненты 🟡🟡🟡',
    'button4': 'Все модификаторы  🔶🔶🔶',
    'searching_button': 'Поиск ',
    'button5': 'Родство🔥💧❄️⚡️🥦',
    'start': 'Меню'
    }

#Обработчик команды старт
@dp.message(CommandStart())
async def process_start_command(message: Message, state: FSMContext):
    #Шоб не было состояний и ошибок
    set_state(message.from_user.id, 'null')
    state.clear()
    #Создаем клавиатуру, передаем число - сколько кнопок в раду, а в строках - callback_data из словаря Knopki
    klava = create_inline_kb(1, 'button1','button2','button3', 'button4','searching_button', 'button5')
    text = '---------Главное меню---------\n\nДанный бот представляет из себя гримуар (справочник) заклинаний-компонентов моды Ars-Magica для Minecraft 1.7.10.'
    await bot.send_message(message.chat.id, text, reply_markup= klava)
#Обработчик нажатия на кнопку Главное меню
@dp.callback_query(F.data == 'start')
async def main_menu(callback: CallbackQuery, state: FSMContext):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    #Шоб не было состояний и ошибок
    set_state(callback.from_user.id, 'null')
    state.clear()
    #Создаем клавиатуру, передаем число - сколько кнопок в раду, а в строках - callback_data из словаря Knopki
    klava = create_inline_kb(1, 'button1','button2','button3', 'button4','searching_button', 'button5')
    text = '---------Главное меню---------\n\nДанный бот представляет из себя гримуар (справочник) заклинаний-компонентов моды Ars-Magica для Minecraft 1.7.10.'
    await bot.send_message(callback.message.chat.id, text, reply_markup= klava)
#Обработчик нажатия на кнопку Все спелы, выводит кнопки со всеми заклинаниями и кнопками перелистывания страниц
@dp.callback_query(F.data == 'button1')
async def everyspell(callback: CallbackQuery, state: FSMContext):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    User_id = callback.from_user.id
    if (proverka_auto(callback.from_user.id, 'all')):
        set_database(callback.from_user.id, 'all')
    #Создаем клавиатуру, передаем число, словарь из базы данных с нужными видами заклинаний, Айди пользователя и Вид заклинания.
    klava = create_spells_buttons(1,get_listing(User_id, 'all'), User_id, 'all')
    #Cохраняем тип просматриваемых заклинаний, чтобы кнопки перелистывания работали корректно.
    set_new_state(callback.from_user.id, 'all')
    set_state(callback.from_user.id, 'all')
    await bot.send_message(callback.message.chat.id, '---------Список заклинаний---------\n\n*Нажмите на номер страницы для того, чтобы вернуться в меню', reply_markup= klava)
#Обработчик нажатия на кнопку << , перелистывает страницу назад, если это не первая страница
@dp.callback_query(F.data == 'backward')
async def back(callback: CallbackQuery, state: FSMContext):
    User_id = callback.from_user.id
    this = await state.get_state()
    page = 0
    type = ''
    #Проверка на то, какие именно заклинания пользователь смотрит на данный момент
    if(get_state(User_id) == 'all'):
        page = get_page(User_id, 'all')
        type = 'all'
        text = '---------Список заклинаний---------\n\n*Нажмите на номер страницы для того, чтобы вернуться в меню'
    elif(get_state(User_id) == 'form'):
        page = get_page(User_id, '1')
        type = '1'
        text = '---------Список форм---------\n\n*Нажмите на номер страницы для того, чтобы вернуться в меню'
    elif(get_state(User_id) == 'comp'):
        page = get_page(User_id, '2')
        type = '2'
        text = '---------Список компонентов---------\n\n*Нажмите на номер страницы для того, чтобы вернуться в меню'
    elif(get_state(User_id) == 'mod'):
        page = get_page(User_id, '3')
        type = '3'
        text = '---------Список модификаторов---------\n\n*Нажмите на номер страницы для того, чтобы вернуться в меню'
    #Проверка на 1 страницу
    if(page > 1):
        await bot.delete_message(callback.message.chat.id, callback.message.message_id)
        update_page(User_id, type, '<<')
        #Создаем клавиатуру, передаем число, словарь из базы данных с нужными видами заклинаний, Айди пользователя и Вид заклинания.
        klava = create_spells_buttons(1,get_listing(User_id, type), User_id, type)
        await bot.send_message(callback.message.chat.id, '---------Список заклинаний---------\n\n*Нажмите на номер страницы для того, чтобы вернуться в меню', reply_markup= klava)
    await callback.answer()
#Обработчик кнопки >>, перелистывает страницу вперед, если это не максимальная страница (В зависимости от типа просмотра)
@dp.callback_query(F.data == 'forward')
async def back(callback: CallbackQuery, state: FSMContext):
    User_id = callback.from_user.id
    this = await state.get_state()
    page = 0
    type = ''
    #Проверка типа просмотра и установка соответсвующих текстов и максимальной страницы
    if(get_state(User_id) == 'all'):
        page = get_page(User_id, 'all')
        type = 'all'
        text = '---------Список заклинаний---------\n\n*Нажмите на номер страницы для того, чтобы вернуться в меню'
        print('ok')
        max = 3
    elif(get_state(User_id) == 'form'):
        page = get_page(User_id, '1')
        type = '1'
        text = '---------Список форм---------\n\n*Нажмите на номер страницы для того, чтобы вернуться в меню'
        max = 1
    elif(get_state(User_id) == 'comp'):
        page = get_page(User_id, '2')
        type = '2'
        text = '---------Список компонентов---------\n\n*Нажмите на номер страницы для того, чтобы вернуться в меню'
        max = 2
    elif(get_state(User_id) == 'mod'):
        page = get_page(User_id, '3')
        type = '3'
        text = '---------Список модификаторов---------\n\n*Нажмите на номер страницы для того, чтобы вернуться в меню'
        max  = 1
    # Проверка на максимальную страницу
    if(page < max):
        await bot.delete_message(callback.message.chat.id, callback.message.message_id)
        update_page(User_id, type, '>>')
        #Создаем клавиатуру, передаем число, словарь из базы данных с нужными видами заклинаний, Айди пользователя и Вид заклинания.
        klava = create_spells_buttons(1,get_listing(User_id, type), User_id, type)
        await bot.send_message(callback.message.chat.id, text, reply_markup= klava)
    await callback.answer()

# Обработчик нажатия на кнопку Все формы, соответственно выволдит все заклинания вида - Форма
@dp.callback_query(F.data == 'button2')
async def formspell(callback: CallbackQuery):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    User_id = callback.from_user.id
    if (proverka_auto(callback.from_user.id, '1')):
        set_database(callback.from_user.id, '1')
    #Создаем клавиатуру, передаем число, словарь из базы данных с нужными видами заклинаний, Айди пользователя и Вид заклинания.
    klava = create_spells_buttons(1,get_listing(User_id, '1'), User_id, '1')
    #Cохраняем тип просматриваемых заклинаний, чтобы кнопки перелистывания работали корректно.
    set_new_state(callback.from_user.id, 'form')
    set_state(User_id, 'form')
    await bot.send_message(callback.message.chat.id, '---------Список форм---------\n*Нажмите на номер страницы для того, чтобы вернуться в меню', reply_markup= klava)
# Обработчик нажатия на кнопку Все компоненты, соответственно выволдит все заклинания вида - Компонент
@dp.callback_query(F.data == 'button3')
async def compspell(callback: CallbackQuery):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    User_id = callback.from_user.id
    if (proverka_auto(callback.from_user.id, '2')):
        set_database(callback.from_user.id, '2')
    #Создаем клавиатуру, передаем число, словарь из базы данных с нужными видами заклинаний, Айди пользователя и Вид заклинания.
    klava = create_spells_buttons(1,get_listing(User_id, '2'), User_id, '2')
    #Cохраняем тип просматриваемых заклинаний, чтобы кнопки перелистывания работали корректно.
    set_new_state(callback.from_user.id, 'comp')
    set_state(User_id, 'comp')
    await bot.send_message(callback.message.chat.id, '---------Список компонентов---------\n*Нажмите на номер страницы для того, чтобы вернуться в меню', reply_markup= klava)
# Обработчик нажатия на кнопку Все Модификаторы, соответственно выволдит все заклинания вида - Модификатор
@dp.callback_query(F.data == 'button4')
async def modspell(callback: CallbackQuery):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    User_id = callback.from_user.id
    if (proverka_auto(callback.from_user.id, '3')):
        set_database(callback.from_user.id, '3')
    #Создаем клавиатуру, передаем число, словарь из базы данных с нужными видами заклинаний, Айди пользователя и Вид заклинания.
    klava = create_spells_buttons(1,get_listing(User_id, '3'), User_id, '3')
    #Cохраняем тип просматриваемых заклинаний, чтобы кнопки перелистывания работали корректно.
    set_new_state(callback.from_user.id, 'mod')
    set_state(User_id, 'mod')
    await bot.send_message(callback.message.chat.id, '---------Список модификаторов---------\n*Нажмите на номер страницы для того, чтобы вернуться в меню', reply_markup= klava)
#Обработчик нажатия на кнопку заклинания, выводит всю информацию о нажатом заклинании.
@dp.callback_query(SpellsCallbackFactory.filter())
async def spell_press(callback: CallbackQuery,
                                 callback_data: SpellsCallbackFactory):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    #Берем инфу о заклинании, айди,фото,название,описание
    id = callback_data.spell_id
    photo =get_photo(id)
    name = callback_data.name
    desc = get_desc(id)
    #Создаем клавиатуру, передаем число - сколько кнопок в раду, а в строках - callback_data из словаря Knopki
    klava = create_inline_kb(1, 'start')
    text = '-------'+name + '-------\n' + desc
    await bot.send_photo(callback.message.chat.id, photo, caption=text, reply_markup=klava)

#Обработчик нажатия на кнопку поиска
@dp.callback_query(F.data == 'searching_button')
async def search(callback: CallbackQuery, state: FSMContext):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    User_id = callback.from_user.id
    #Создаем клавиатуру, передаем число - сколько кнопок в раду, а в строках - callback_data из словаря Knopki
    klava = create_inline_kb(1, 'start')
    #Запрашиваем ввод поискового текста
    await state.set_state(FSMFillForm.search)
    await bot.send_message(callback.message.chat.id, 'Пожалуйста, введите название заклинания с большой буквы (Не меньше 4 символов)', reply_markup= klava)
#Обработчик поискового текста, он срабатывает только если бот ждет поиск, а он его будет ждать.
@dp.message(StateFilter(FSMFillForm.search))
async def search_result(message: Message, state: FSMContext):
    #Создаем клавиатуру, передаем число - сколько кнопок в раду, а в строках - callback_data из словаря Knopki
    klava = create_inline_kb(1, 'start')
    #Проверка на то, что символов больше 4
    if (len(message.text) <= 4):
        await bot.send_message(message.chat.id, 'Пожалуйста, введите название заклинания с большой буквы (Не меньше 4 символов)\n\nМожете попытаться ввести еще раз', reply_markup= klava)
    else:
            #Сохраняем в name результат первого подходящего заклинания.
            name = searching_database(message.text)
            #Проверка на пустой ответ, нужно,чтобы при неудавшемся поиске оповестить пользователя
            if (name == "None" or name is None):
                await bot.send_message(message.chat.id, 'Такое заклинание не было найдено(\n\nМожете попытаться ввести еще раз', reply_markup= klava)
            else:
                #Достаем айди, описание, фото и текст найденного заклинания по названию и далее по айди.
                id = searching_by_name(name)
                desc = get_desc(id)
                photo = get_photo(id)
                text = '-------'+name + '-------\n' + desc
                await bot.send_photo(message.chat.id, photo, caption=text, reply_markup=klava)
                #Перестаем ждать отправки поискового текста.
                state.clear()

#Обработчик нажатия на кнопку Родство, выводит базовую информацию о свойствах каждого родства.
@dp.callback_query(F.data == 'button5')
async def rodstvo(callback: CallbackQuery):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    #Создаем клавиатуру, передаем число - сколько кнопок в раду, а в строках - callback_data из словаря Knopki
    klava = create_inline_kb(1, 'start')
    text = '''
✡️✡️✡️Arcane(Тайна):\nВаши заклинания стоят на 5% меньше маны. Также, при 100% у вас есть шанс, что вы получите особый эффект, под действием которого следующее заклинание будет стоить 0 маны. Однако вы будете получать больше магического урона.\n\n
🪨🪨🪨 Earth(Земля):\nПри 100% родства вы получаете на 10% меньше урона. Маги земли становятся тяжелыми, им трудно плавать в воде и они не могут не получить урон от падения, спрыгнув с небольшой высоты.\n\n
☁️☁️☁️Air(Воздух):\nНа 50% родства вы можете прыгать на 1 блок выше и падать с высоты выше на 1 блок. Этот бонус растет до 2 на 75% и до 3 на 100%. Также, маги воздуха не любят дожди, потому что их постоянно носит ветром во все стороны. Чтобы это предотвратить, нужно ходить на Shift`e.\n\n
🔥🔥🔥Fire(Огонь):\nПри 100% родства вы получаете на 60% меньше урона от огня, а также поджигаете цель при ударе. Начиная с 50% вы получаете урон от воды, но не ниже 75% здоровья.\n\n
💧💧💧Water(Вода):\nВодные маги быстрее плавают, дольше находятся под водой, их не сносит течением воды. 100% родство дает возможность сопротивляться эндерменам, ведь они получат урон, когда нападут на вас. Однако, маги воды получают периодический урон в аду, но не ниже 75% здоровья.\n\n
🥦🥦🥦Nature(Природа):\nРодство с природой дает вам дополнительные саженцы и семена при сборе растений. При 50% родства вы получаете эффект шипов, как чары для брони, и урон от них возрастает до полного родства. При 100% вы будете способны карабкаться по стенам, подобно паукам, а также медленно пополнять голод на солнечном свете. Но маги природы двигаются медленнее из-за корней, при 100% вы начнете намного быстрее терять голод ночью или в темных местах.\n\n
❄️❄️❄️Ice(Холод):\nПри 50% родства вы можете замораживать воду в лёд, когда крадетесь, а при 100% - лаву в обсидиан. Однако вы двигаетесь медленно, если находитесь не на льду.\n\n
⚡️⚡️⚡️Lighting(Электричество):\nСкорость увеличивается до 50% при полном родстве, также на 50% вы можете взбираться на 1 блок в высоту без прыжка. На 50% вы начнете быстро терять ману под дождем или в воде. На 75% вы можете зажигать стоящие рядом динамиты или наэлектризовывать криперов, а также ваше максимальное здоровье в воде уменьшается до 6 сердец. На 100% эти эффекты пропадают, и вы только теряете ману в воде и под дождем. Также, на 100% при ударе моба пустой рукой - в него ударит молния!\n\n
❤️❤️❤️Life(Жизнь):\nЧем выше ваше родство, тем быстрее вы восстанавливаете здоровье, независимо от уровня голода. Однако каждый раз, когда вы кого-то убиваете, вы получаете эффект слабости и тошноты.\n\n
⚫️⚫️⚫️Ender(Эндер):\nПри 50% родства вы получаете защиту от яда, при 75% - ночное зрение, при 100% - эндермены не будут вас атаковать первыми. Однако ваше максимальное здоровье будет уменьшено на свету, а при более высоком уровне родства - и в воде. Мастера магии эндера перебороли эту слабость.\n\n

'''
    await bot.send_message(callback.message.chat.id, text, reply_markup= klava)


#Функция для быстрого создания кнопочек
def create_inline_kb(width: int,
                     *args: str,
                     **kwargs: str) -> InlineKeyboardMarkup:
    kb_builder = InlineKeyboardBuilder()
    buttons: list[InlineKeyboardButton] = []
    #Если передан список
    if args:
        for button in args:
            buttons.append(InlineKeyboardButton(
                text=KNOPKI[button] if button in KNOPKI else button,
                callback_data=button))
    #Если передан словарь
    if kwargs:
        for button, text in kwargs.items():
            buttons.append(InlineKeyboardButton(
                text=text,
                callback_data=button))
    kb_builder.row(*buttons, width=width)
    #Возвращает клавиатуру
    return kb_builder.as_markup()

#Функция, которая записывает нового пользователя в базу с типом просмотра.
def set_database(id,type):
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    cursor.execute(f"INSERT INTO users (id, type, page) VALUES ({id}, '{type}', 1)")
    connection.commit()
    connection.close()
#Функция, которая проверяет, есть ли пользователь  в базе с таким типом просмотра.
def proverka_auto(id, type):
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT id, type FROM users WHERE id = {id} AND type = '{type}'")
    result = cursor.fetchone()
    connection.close()
    if (result == None):
        return True
    else:
        return False
# Функция, которая возвращает словарь нужных нам заклинаний, в зависимости от страницы и типа просмотра.
def get_listing(id, type):
    page = get_page(id,type)
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    result = f"SELECT id, name FROM spells WHERE id >({page}-1)*10 AND id <= {page}*10"
    if (type == '1'):
        result = f"SELECT id, name FROM spells WHERE category = 1"
    elif (type == '2' and page == 1):
        result = f"SELECT id, name FROM spells WHERE id >({page}-1)*10 AND id <= {page}*18 AND category = 2"
    elif (type == '2' and page == 2):
        result = f"SELECT id, name FROM spells WHERE id >({page}-1)*18 AND id <= {page}*30 AND category = 2"
    elif (type == '3'):
        result = f"SELECT id, name FROM spells WHERE category = 3"

    cursor.execute(result)
    result = cursor.fetchall()
    connection.close()
    #Переводим список кортежей в словарь
    diction = {key: value for key, value in result}
    return diction
#Функция которая создает кнопочки из словаря с заклинаниями.
def create_spells_buttons(width: int, spells_dic,id, type):
    page = get_page(id,type)
    kb_builder = InlineKeyboardBuilder()
    buttons: list[InlineKeyboardButton] = []
    for key, value in spells_dic.items():
        buttons.append(InlineKeyboardButton(
            text = str(value),
            callback_data= SpellsCallbackFactory(
                spell_id = key,
                name= value
            ).pack()
        )
        )
    #Вычисляем максимум страниц в зависимости от типа просмотра, нужно для средней кнопочки с номером страницы.
    if (type == '1'):
        max = 1
    elif (type == '2'):
        max = 2
    elif (type == '3'):
        max = 1
    else:
        max = 3
    #Кнопка перелистывания страницы назад
    buttons.append(InlineKeyboardButton(text = '<<', callback_data= 'backward'))
    #Создаем билдер клавиатуры из списка кнопочек
    kb_builder.row(*buttons, width=width)
    #Добавляем дополнительно кнопки текущей страницы и кнопки перелистывания вперед.
    kb_builder.add(InlineKeyboardButton(text = f'{page}/{max}', callback_data='start'),InlineKeyboardButton(text='>>', callback_data='forward'))
    return kb_builder.as_markup()
#Функкция для получения текущей страницы пользователя из базы
def get_page(id, type):
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT page FROM users WHERE id = {id} AND type = '{type}'")
    result = cursor.fetchone()[0]
    connection.close()
    return result
#Меняет страницу пользователя в засимости от типа и стрелки (Вперед или назад)
def update_page(id,type, strelka):
    page = get_page(id,type)
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    if(strelka == ">>"):
        page = page + 1
        cursor.execute(f"UPDATE users SET page = {page} WHERE id = {id} AND type = '{type}'")
        print(1)
    elif(strelka == "<<"):
        page = page - 1
        cursor.execute(f"UPDATE users SET page = {page} WHERE id = {id} AND type = '{type}'")
        print(2)
    connection.commit()
    connection.close()

#Получает описание заклинания из базы
def get_desc(id):
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT desc FROM spells WHERE id = {id}")
    result = cursor.fetchone()[0]
    connection.close()
    return result
#Получает url адрес фотки заклинания
def get_photo(id):
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT url_photo FROM spells WHERE id = {id}")
    result = cursor.fetchone()[0]
    connection.close()
    return result
#Получает статус текущего просмотра, тип
def get_state(id):
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT state FROM states WHERE id = {id}")
    result = cursor.fetchone()[0]
    connection.close()
    return result
#Устанавливает нужный нам статус просмотра типа заклинаний.
def set_state(id, state):
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    cursor.execute(f"UPDATE states SET state = '{state}' WHERE id = {id}")
    connection.commit()
    connection.close()
#Записывает нового пользователя в базу статусов просмотра типа, если его там нет.
def set_new_state(id,state):
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    cursor.execute(f"INSERT OR IGNORE INTO states(id,state) VALUES ({id}, '{state}')")
    connection.commit()
    connection.close()
#Ищет Подстроку text среди названий заклинаний. Возвращает название найденного заклинания.
def searching_database(text):
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT id,name FROM spells")
    result = cursor.fetchall()
    connection.close()
    for i in result:
        value = i[1]
        if (text in value):
            return value
    return 'None'
#Получает айди по названию заклинания.
def searching_by_name(text):
    connection = sqlite3.connect('ars_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT id FROM spells WHERE name = '{text}'")
    result = cursor.fetchone()[0]
    connection.close()
    return result
# Запускаем диспетчер для отловки обновлений.
if __name__ == '__main__':
    dp.run_polling(bot)


##### Библиотекии
1) Aiogram - Библиотека для написания телеграмм ботов. (https://docs.aiogram.dev/en/latest/)
2) Nest_asyncio - исправляет ошибочки (https://pypi.org/project/nest-asyncio/)
3) Wget - Качает базу данных с интернета (https://www.gnu.org/software/wget/manual/wget.html#URL-Format)
4) sqlite3 - Работает с базой данных (https://www.sqlite.org/docs.html)